In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)
from tqdm import tqdm
tqdm.pandas(desc='pandas bar')
import pickle


In [2]:
data_path = r'./data_for_ctr_predict/'
with open(data_path+'doc_info.txt', 'r') as f:
    lines = f.readlines()

item = []
for line in tqdm(lines):
    unit = line.split('\t')
    if len(unit)<7:
        for _ in range(7-len(unit)):
            unit.append(np.nan)
    item.append(unit)
#item = np.array(item)
item = pd.DataFrame(item, columns=['item_id', 'item_title', 'item_time', 'item_picture', 'item_cluster1', 'item_cluster2', 'item_keywords'])
item['item_id'] = item['item_id'].astype('int64')

100%|██████████| 633391/633391 [00:02<00:00, 289165.89it/s]


# 对每个doc的title使用Bert进行embedding

In [3]:
# 构建tokenizer和model
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModel.from_pretrained("bert-base-chinese")

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
import torch
title_list = item.item_title.values

model.to('cuda:2')
res = torch.tensor([])

bs = 1000
with torch.no_grad():
    for i in tqdm(range(int(title_list.shape[0] / bs) + 1)):
        batch = title_list[i * bs: (i + 1) * bs]
        input = tokenizer(list(batch), padding=True, truncation=True, max_length=512, return_tensors="pt")
        input.to('cuda:2')
        output = model(**input)
        res = torch.cat([res, output['pooler_output'].cpu().detach()], dim=0)

100%|██████████| 633/633 [07:33<00:00,  1.40it/s]


In [13]:
res.shape

torch.Size([633391, 768])

In [14]:
res = res.cpu().detach().numpy()

In [18]:
with open(r'title_768.pkl', 'wb') as f:
    pickle.dump(res, f, pickle.HIGHEST_PROTOCOL)

In [3]:
with open(r'title_768.pkl', 'rb') as f:
    res = pickle.load(f)

In [4]:
# 做降维
from sklearn.decomposition import PCA

n_dim = 16
transformer = PCA(n_components=n_dim)
res = transformer.fit_transform(res)

In [ ]:
with open(r'title_' + str(n_dim) + '.pkl', 'wb') as f:
    pickle.dump(res, f, pickle.HIGHEST_PROTOCOL)